In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, Pipeline

2023-11-11 10:32:49.121923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 10:32:49.984493: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-11 10:32:49.984654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-11 10:32:49.984661: W tensorfl

In [2]:
transformers.__version__

'4.35.0'

In [3]:
# Install transformers 4.35.0 -> earlier versions have problem with zephir (KeyError mistral)
#!pip install transformers -U

In [16]:
def in_context_prompt(question, context):
    return f"""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

                {context}

                Question: {question}"""


class LLMPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        # In context learning if Q&A are given
        if "question" in kwargs and "context" in kwargs:
            preprocess_kwargs["question"] = kwargs["question"]
            preprocess_kwargs["context"] = kwargs["context"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, question=None, context=None):
        # Create in context prompt
        if question and context:
            inputs = in_context_prompt(question=question, context=context)
        
        # Tokenize input
        model_inputs = self.tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')
        
        return model_inputs

    def _forward(self, model_inputs):
        output_tokens = self.model(**model_inputs)
        return output_tokens

    def postprocess(self, output_tokens):
        outputs = self.tokenizer.convert_tokens_to_string(output_tokens)
        return outputs

In [ ]:
model_id = "HuggingFaceH4/zephyr-7b-alpha"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
# Probably not possible to be flexible in pipeline